In [165]:
import simpy
import itertools
import numpy as np

## SINGLE-SERVER QUEUEING SYSTEM

Processes
- arrival 
- service

#### Generating all cars

In [157]:
def car(env, name, bcs, driving_time, charge_duration):
    # Simulate driving to Charging Station
    yield env.timeout(driving_time)
    
    # Request one of it's charging stations
    print('car {} is arriving at {:.0f}'.format(name, env.now))
    with bcs.request() as req:
        yield req
        
        # Charge the battery
        print('starting to charge Car {} at {}'.format(name, env.now))
        yield env.timeout(charge_duration)
        print('Car {} finished charging at {}'.format(name, env.now))
    

In [159]:
env = simpy.Environment()
bcs = simpy.Resource(env, capacity=2)
env.process(car(env, "1", bcs, 1, 5))
env.process(car(env, "2", bcs, 2, 5))
env.process(car(env, "3", bcs, 3, 5))
env.process(car(env, "4", bcs, 4, 5))
env.run(until=60)

car 1 is arriving at 1
starting to charge Car 1 at 1
car 2 is arriving at 2
starting to charge Car 2 at 2
car 3 is arriving at 3
car 4 is arriving at 4
Car 1 finished charging at 6
starting to charge Car 3 at 6
Car 2 finished charging at 7
starting to charge Car 4 at 7
Car 3 finished charging at 11
Car 4 finished charging at 12


#### with car generator

In [173]:
def car(env, name, queue, charge_duration):
    print('{} is arriving at {:.2f}'.format(name, env.now))
    with queue.request() as req:
        yield req
        
        # Charge the battery
        print('starting to serve {} at {:.2f}'.format(name, env.now))
        yield env.timeout(np.random.exponential(charge_duration))
        print('{} finished serving at {:.2f}'.format(name, env.now))
        

In [174]:
def car_generator(env, queue):
    interarrival_time = 10
    charge_duration = 5
    """Generate new cars that arrive at the gas station."""
    for i in itertools.count():
#     for i in range(1,5):
        yield env.timeout(np.random.exponential(10))
        
        env.process(car(env, 'Car %d' % i, queue, charge_duration ))
    return i

In [175]:
env = simpy.Environment()
queue = simpy.Resource(env, capacity=2)
proc = env.process(car_generator(env, queue))
env.run(until=60)

Car 0 is arriving at 6.77
starting to serve Car 0 at 6.77
Car 1 is arriving at 9.81
starting to serve Car 1 at 9.81
Car 1 finished serving at 11.48
Car 2 is arriving at 13.19
starting to serve Car 2 at 13.19
Car 3 is arriving at 14.31
Car 2 finished serving at 16.00
starting to serve Car 3 at 16.00
Car 0 finished serving at 16.57
Car 3 finished serving at 19.04
Car 4 is arriving at 46.24
starting to serve Car 4 at 46.24
Car 4 finished serving at 47.55
Car 5 is arriving at 49.50
starting to serve Car 5 at 49.50
Car 6 is arriving at 49.68
starting to serve Car 6 at 49.68
Car 7 is arriving at 52.68
Car 8 is arriving at 52.76
Car 5 finished serving at 54.19
starting to serve Car 7 at 54.19
Car 7 finished serving at 54.64
starting to serve Car 8 at 54.64
Car 9 is arriving at 55.02
Car 6 finished serving at 58.51
starting to serve Car 9 at 58.51


In [169]:
np.mean(np.random.exponential(10,size=100))

9.735241301868783